In [1]:
#Install pip packages in the curret Jupyter kernal
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install --upgrade pyodbc
!{sys.executable} -m pip install polars

  Using cached pip-23.2-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ---------------------------------------- 0.0/69.7 kB ? eta -:--:--
     ---------------------------------------- 69.7/69.7 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: pyodbc
    Found existing installation: pyodbc 4.0.34
    Uninstalling pyodbc-4.0.34:
      Successfully uninstalled pyodbc-4.0.34
  Obtaining dependency information for polars from https://files.pythonhosted.org/packages/c5/18/4f0fbacc2f04eb1ef64cb7fe452d849d32c17bb10543391cfe7ac0cd4f9b/polars-0.18.7-cp38-abi3-win_amd64.whl.metadata
  Using cached polars-0.18.7-cp38-abi3-win_amd64.whl.metadata (14 kB)
Using cached polars-0.18.7-cp38-abi3-win_amd64.whl (19.8 MB)


In [150]:
def pop_random(lst):
    idx = random.randrange(0, len(lst))
    return lst.pop(idx)

wld_list = ['W'] * 45 + ['L'] * 45 + ['D'] * 10

def match_dict(n):
    lst = ["Player " + str(i) for i in range(n)]
    pairs = []
    pair_dict = {}
    point_dict = {}
    while lst:
        rand1 = pop_random(lst)
        rand2 = pop_random(lst)
        pair = [rand1, rand2]
        pairs.append(pair)
    for pair in pairs:
        pair_dict[pair[0]] = pair[1]
        pair_dict[pair[1]] = pair[0]
        wld = random.choice(wld_list)
        if wld == "W":
            point_dict[pair[0]] = 3
            point_dict[pair[1]] = 0
        elif wld == "L":
            point_dict[pair[0]] = 0
            point_dict[pair[1]] = 3
        elif wld == "D":
            point_dict[pair[0]] = 1
            point_dict[pair[1]] = 1
        
    return pair_dict, point_dict

Notes on behavior:

Byes do not count towards OMW and OGW one way or othe other.

If a bye is assigned, it's first assigned among subjects with the fewest byes (usually zero) and second among subjects with the fewest points (should be updated to lowest overall ranking).

I've calculated omw, gw, and ogw wrong. They're fractions of points, not fractions of wins. See https://media.wizards.com/2023/wpn/marketing_materials/wpn/mtg_mtr_2023may29_en.pdf



In [1]:
import random
import polars as pl
import networkx as nx

In [182]:
players = ["Esben",
          "Jeppe",
          "Halfdan",
          "Asger",
          "Bakke"]
if (len(players) > len(set(players))):
    raise ValueError('There are non-unique players.')
elif ("Bye" in players):
    raise ValueError('You have listed "Bye" as a player. Please don\'t.')

players_df = pl.DataFrame({"player": players, "match_pts": 0, "game_pts": 0, "match_tot": 0, "game_tot": 0, "opponents": "", "dropped": False, "mw": 0., "gw": 0., "omw": 0., "ogw": 0., "grp_idx": 0})
players_df

player,match_pts,game_pts,match_tot,game_tot,opponents,dropped,mw,gw,omw,ogw,grp_idx
str,i64,i64,i64,i64,str,bool,f64,f64,f64,f64,i64
"""Esben""",0,0,0,0,"""""",false,0.0,0.0,0.0,0.0,0
"""Jeppe""",0,0,0,0,"""""",false,0.0,0.0,0.0,0.0,0
"""Halfdan""",0,0,0,0,"""""",false,0.0,0.0,0.0,0.0,0
"""Asger""",0,0,0,0,"""""",false,0.0,0.0,0.0,0.0,0
"""Bakke""",0,0,0,0,"""""",false,0.0,0.0,0.0,0.0,0


In [191]:
def pairings(pair_players):
    
    #Remove dropped players from pairing considerations
    pair_players = pair_players.filter(pl.col("dropped") == False)
    
    #If a bye needs to be assigned, do so first.
    has_bye = len(pair_players) % 2 == 1
    
    if (has_bye):
        #Find all players with the lowest number of byes (usually 0) and the highest group index in this group (lowest number of match points, poorest [o](m/g)w)
        bye_players = pair_players.with_columns(
            bye_count = pl.col("opponents").str.count_match("Bye")
        ).filter(
            pl.col("bye_count") == pl.col("bye_count").min()
        ).filter(
            pl.col("grp_idx") == pl.col("grp_idx").max()
        )
        
        bye_pair = bye_players.sample(1).select("player").vstack(
                pl.DataFrame({"player": "Bye"})
            ).with_columns(
                match = 0,
                play_draw = pl.Series(["play", "draw"])
            )
        
        pair_players = pair_players.join(bye_pair, on = "player", how = "anti").with_columns(forbidden = pl.lit(""))
                
    pairs_n = len(pair_players)/2
    
    pairs = pair_players.select("player").with_columns(
        match = 0,
        play_draw = pl.lit("play")
    ).clear()
    
    #Extract list of opponents and assign a weight based on rank group
    pair_players = pair_players.with_columns(
        oppo_list = pl.col("opponents").str.split("//"),
        #grp_pts = 1/(1+pl.col("grp_idx")),
        grp_pts = 1 + pl.col("grp_idx").max() - pl.col("grp_idx")
    )
    
    graph_players = dict(zip(pair_players["player"].to_list(), pair_players["oppo_list"].to_list()))
    graph_points = dict(zip(pair_players["player"].to_list(), pair_players["grp_pts"].to_list()))

    bracketGraph = nx.Graph()
    bracketGraph.add_nodes_from(graph_players)
    
    #Core pairing loop. Becomes slow with > 100 participants. Consider splitting it into piece when groups are big.
    for player in bracketGraph.nodes():
        for opponent in bracketGraph.nodes():
            if opponent not in graph_players[player] and player != opponent:
                #wgt = random.randint(1, 9) #randomizes pairings. Not necessarily desirable.
                wgt = 1 + (graph_points[player] + graph_points[opponent])**2
                bracketGraph.add_edge(player, opponent, weight=wgt)
    
    pairings_bg = nx.max_weight_matching(bracketGraph)
    
    for idx, pair_bg in enumerate(pairings_bg):
        pair_df = pl.DataFrame({"player": pair_bg, "match": idx+1}).with_columns(
            match = pl.col("match").cast(pl.Int32),
            play_draw = pl.Series(random.sample(["play", "draw"], 2))
        )
        pairs = pairs.vstack(pair_df)

    if (has_bye):
        pairs = bye_pair.vstack(pairs)
            
    pairs = pairs.pivot(
        values="player", index="match", columns="play_draw", aggregate_function="first"        
    ).select(["match", "play", "draw"])
    
    return pairs

In [184]:
pairings_1 = pairings(players_df)
pairings_1

match,play,draw
i32,str,str
0,"""Halfdan""","""Bye"""
1,"""Jeppe""","""Esben"""
2,"""Bakke""","""Asger"""


In [195]:
def results(players_df, pairings_df, result_list = None, drop_list = None):
    
    #TODO: Accept manual results
    if (result_list is None):
        result_list = input("How did it go? ")
        
    if (len(pairings_df.filter(pl.col("play") == "Bye")) > 0):
        result_list = ["0-2"] + result_list
    elif (len(pairings_df.filter(pl.col("draw") == "Bye")) > 0):
        result_list = ["2-0"] + result_list
    
    result_df = pl.DataFrame({"result": result_list}).with_columns(
        wins = pl.col("result").str.split_exact("-", 2)
    ).unnest("wins").rename(
        {"field_0": "play_wins", "field_1": "draw_wins", "field_2": "games_drawn"}
    ).with_columns(
        play_wins = pl.col("play_wins").cast(pl.Int32),
        draw_wins = pl.col("draw_wins").cast(pl.Int32),
        games_drawn = pl.col("games_drawn").cast(pl.Int32),
        play_points = pl.when(pl.col("play_wins") > pl.col("draw_wins")).then(pl.lit(3))\
            .when(pl.col("play_wins") == pl.col("draw_wins")).then(pl.lit(1))\
            .otherwise(pl.lit(0)),
        draw_points = pl.when(pl.col("draw_wins") > pl.col("play_wins")).then(pl.lit(3))\
            .when(pl.col("draw_wins") == pl.col("play_wins")).then(pl.lit(1))\
            .otherwise(pl.lit(0))
    ).fill_null(0).with_columns(
        total_games = pl.col("play_wins") + pl.col("draw_wins") + pl.col("games_drawn")
    )
    
    result_df = pairings_df.hstack(result_df)
    
    play_df = result_df.select(pl.col("^play_.*$"), player = "play", new_opponent = "draw", new_draws = "games_drawn", new_games = "total_games")
    draw_df = result_df.select(pl.col("^draw_.*$"), player = "draw", new_opponent = "play", new_draws = "games_drawn", new_games = "total_games")

    play_df.columns = list(map(lambda x: x.replace("play_", "new_"), play_df.columns))
    draw_df.columns = list(map(lambda x: x.replace("draw_", "new_"), draw_df.columns))
    
    update_df = play_df.vstack(draw_df).filter(pl.col("player") != "Bye")
    
    players_df = players_df.drop(["gw", "omw", "ogw", "grp_idx"]).join(update_df, on = "player", how = "left").with_columns(
        match_pts = pl.col("match_pts") + pl.col("new_points"),
        game_pts = pl.col("game_pts") + 3*pl.col("new_wins") + pl.col("new_draws"),
        match_tot = pl.col("match_tot") + pl.lit(1),
        game_tot = pl.col("game_tot") + pl.col("new_games"),
        opponents = (pl.col("opponents") + pl.lit("//") + pl.col("new_opponent")).str.replace("^//", "")
    ).select(pl.col("^[^n].*$")).with_columns(
        oppo_struct = pl.col("opponents").str.split("//").list.to_struct()
    ).with_columns(
        mw = 100*pl.col("match_pts")/(3*pl.col("match_tot")),
        gw = 100*pl.col("game_pts")/(3*pl.col("game_tot"))
    ).with_columns(
        mw = pl.when(pl.col("mw")>= 33).then(pl.col("mw")).otherwise(33),
        gw = pl.when(pl.col("gw")>= 33).then(pl.col("gw")).otherwise(33)
    )
    
    player_opponent_df = players_df.unnest("oppo_struct").select(pl.col("^field.*$"), "player").melt(
        id_vars = "player", value_vars = None, value_name = "opponent"
    ).drop("variable").join(
        players_df.select(["player", "mw", "gw"]), left_on = "opponent", right_on = "player", how = "inner"
    ).groupby("player").mean().drop("opponent").rename(
        {"mw": "omw", "gw": "ogw"}
    )
    
    players_df = players_df.join(
        player_opponent_df, on = "player", how = "left"
    ).fill_null(33).sort(["match_pts", "omw", "gw", "ogw"], descending = True).drop("oppo_struct").with_columns(
        dropped = pl.when(pl.col("player").is_in(drop_list)).then(pl.lit(True)).otherwise(pl.lit(False))
    )
    
    #Create an index that encapsulates ranking information.
    grp_idx = []
    for idx, df in enumerate(players_df.groupby(["match_pts", "omw", "gw", "ogw"], maintain_order = True)):
        grp_idx = grp_idx + [idx] * len(df[1])
    players_df = players_df.hstack(pl.DataFrame({"grp_idx": grp_idx}))

    return players_df

In [196]:
players_df_1 = results(players_df, pairings_1, ["2-1", "1-1-1"])
pairings_2 = pairings(players_df_1)
print(pairings_1)
players_df_1

shape: (3, 3)
┌───────┬─────────┬───────┐
│ match ┆ play    ┆ draw  │
│ ---   ┆ ---     ┆ ---   │
│ i32   ┆ str     ┆ str   │
╞═══════╪═════════╪═══════╡
│ 0     ┆ Halfdan ┆ Bye   │
│ 1     ┆ Jeppe   ┆ Esben │
│ 2     ┆ Bakke   ┆ Asger │
└───────┴─────────┴───────┘


player,match_pts,game_pts,match_tot,game_tot,opponents,dropped,mw,gw,omw,ogw,grp_idx
str,i64,i64,i64,i64,str,bool,f64,f64,f64,f64,i64
"""Halfdan""",3,6,1,2,"""Bye""",false,100.0,100.0,33.0,33.0,0
"""Jeppe""",3,6,1,3,"""Esben""",false,100.0,66.666667,33.0,33.333333,1
"""Asger""",1,4,1,3,"""Bakke""",false,33.333333,44.444444,33.333333,44.444444,2
"""Bakke""",1,4,1,3,"""Asger""",false,33.333333,44.444444,33.333333,44.444444,2
"""Esben""",0,3,1,3,"""Jeppe""",false,33.0,33.333333,100.0,66.666667,3


In [197]:
players_df_2 = results(players_df_1, pairings_2, ["2-1", "1-1-1"])
pairings_3 = pairings(players_df_2)
print(pairings_3)
players_df_2

shape: (3, 3)
┌───────┬─────────┬───────┐
│ match ┆ play    ┆ draw  │
│ ---   ┆ ---     ┆ ---   │
│ i32   ┆ str     ┆ str   │
╞═══════╪═════════╪═══════╡
│ 0     ┆ Asger   ┆ Bye   │
│ 1     ┆ Halfdan ┆ Esben │
│ 2     ┆ Jeppe   ┆ Bakke │
└───────┴─────────┴───────┘


player,match_pts,game_pts,match_tot,game_tot,opponents,dropped,mw,gw,omw,ogw,grp_idx
str,i64,i64,i64,i64,str,bool,f64,f64,f64,f64,i64
"""Jeppe""",4,10,2,6,"""Esben//Asger""",false,66.666667,55.555556,41.666667,52.222222,0
"""Bakke""",4,10,2,6,"""Asger//Halfdan…",false,66.666667,55.555556,41.666667,52.222222,0
"""Halfdan""",3,9,2,5,"""Bye//Bakke""",false,50.0,60.0,66.666667,55.555556,1
"""Esben""",3,9,2,5,"""Jeppe//Bye""",false,50.0,60.0,66.666667,55.555556,1
"""Asger""",2,8,2,6,"""Bakke//Jeppe""",false,33.333333,44.444444,66.666667,55.555556,2


In [180]:
players_df_2

player,match_pts,game_pts,match_tot,game_tot,opponents,dropped,mw,gw,omw,ogw,grp_idx
str,i64,i64,i64,i64,str,bool,f64,f64,f64,f64,i64
"""Halfdan""",6,12,2,6,"""Asger//Bakke""",false,100.0,66.666667,50.0,60.0,0
"""Bakke""",3,9,2,5,"""Bye//Halfdan""",false,50.0,60.0,100.0,66.666667,1
"""Asger""",3,9,2,5,"""Halfdan//Bye""",false,50.0,60.0,100.0,66.666667,1
"""Esben""",2,12,2,6,"""Jeppe//Jeppe""",false,33.333333,66.666667,33.333333,66.666667,2
"""Jeppe""",2,12,2,6,"""Esben//Esben""",false,33.333333,66.666667,33.333333,66.666667,2


In [181]:
results(players_df_2, pairings_3, ["2-1", "1-1-1"])

player,match_pts,game_pts,match_tot,game_tot,opponents,dropped,mw,gw,omw,ogw,grp_idx
str,i64,i64,i64,i64,str,bool,f64,f64,f64,f64,i64
"""Halfdan""",9,18,3,9,"""Asger//Bakke//…",false,100.0,66.666667,37.037037,54.166667,0
"""Esben""",5,18,3,8,"""Jeppe//Jeppe//…",false,55.555556,75.0,33.333333,66.666667,1
"""Asger""",4,15,3,8,"""Halfdan//Bye//…",false,44.444444,62.5,66.666667,66.666667,2
"""Bakke""",3,12,3,8,"""Bye//Halfdan//…",false,33.333333,50.0,100.0,66.666667,3
"""Jeppe""",3,18,3,9,"""Esben//Esben//…",false,33.333333,66.666667,51.851852,70.833333,4
